### DQN using LIDAR dataset

In [ ]:
import gymnasium as gym
import torch
import numpy as np
from stable_baselines3.common.env_checker import check_env
from metadrive.envs import MetaDriveEnv
from metadrive.engine.engine_utils import close_engine

# --- SeedWrapper ---
class SeedWrapper(gym.Wrapper):
    def reset(self, seed=None, **kwargs):
        return self.env.reset(**kwargs)

# --- DiscreteActionWrapper ---
class DiscreteActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        # Define a fixed set of discrete actions for MetaDrive
        # MetaDrive actions: [steering, throttle, brake]
        self.discrete_actions = [
            np.array([0.0, 1.0, 0.0]),   # accelerate
            np.array([0.0, 0.0, 0.0]),   # no-op
            np.array([-0.5, 1.0, 0.0]),  # turn left while accelerating
            np.array([0.5, 1.0, 0.0]),   # turn right while accelerating
            np.array([0.0, 0.0, 1.0]),   # brake
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))
        
    def action(self, action_idx):
        return self.discrete_actions[action_idx]
    
    def reset(self, **kwargs):
        kwargs.pop("options", None)
        return self.env.reset(**kwargs)

# --- LidarOnlyObservationWrapper ---
class LidarOnlyObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, lidar_points=240):
        super().__init__(env)
        self.lidar_points = lidar_points
        # Define the observation space for LIDAR data
        self.observation_space = gym.spaces.Box(low=0.0, high=50.0, shape=(lidar_points,), dtype=np.float32)
    
    def observation(self, obs):
        # Expect obs to be a flat NumPy array
        if not isinstance(obs, np.ndarray):
            raise ValueError(f"Observation must be a NumPy array. Got: {type(obs)}")
        
        # Debug: Print raw observation shape and type
        print(f"Raw observation type: {type(obs)}, shape: {obs.shape}")
        
        # Ensure the data is a 1D array
        if obs.ndim > 1:
            obs = obs.flatten()
        
        # Check if the observation size matches expected LIDAR points
        if obs.shape[0] != self.lidar_points:
            print(f"Warning: Observation size {obs.shape[0]} doesn’t match expected {self.lidar_points}. Truncating to LIDAR data.")
            obs = obs[:self.lidar_points]  # Assume first 240 elements are LIDAR
        
        # Convert to float32 array and clip to valid range
        lidar_obs = np.array(obs, dtype=np.float32)
        lidar_obs = np.clip(lidar_obs, 0.0, 50.0)
        return lidar_obs

# Example usage with MetaDrive and rendering
if __name__ == "__main__":
    # Ensure the engine is closed from any previous runs
    close_engine()  # Explicitly close any existing engine instance
    
    # Configure the MetaDrive environment with rendering enabled
    config = {
        "use_render": True,           # Enable 3D rendering for visualization
        "manual_control": False,      # No human control; agent drives
        "traffic_density": 0.1,       # Sparse traffic for simplicity
        "num_scenarios": 1,           # Single scenario
        "start_seed": 42,             # Fixed seed for reproducibility
        "vehicle_config": {
            "lidar": {"num_lasers": 240, "distance": 50.0},  # Enable LIDAR with 240 points
            "side_detector": {"num_lasers": 0},              # Minimize SideDetector output
            "lane_line_detector": {"num_lasers": 0}          # Minimize LaneLineDetector output
        },
        "image_observation": False    # Disable image observations; use LIDAR
    }
    
    # Create the environment and apply wrappers
    env = MetaDriveEnv(config)
    env = SeedWrapper(env)
    env = DiscreteActionWrapper(env)
    env = LidarOnlyObservationWrapper(env, lidar_points=240)
    
    # Check the environment for Gym compatibility
    check_env(env)
    
    # Training loop with visualization
    num_episodes = 500  # Number of episodes to train/visualize
    max_steps = 200   # Max steps per episode
    
    for episode in range(num_episodes):
        obs, info = env.reset()  # Unpack the tuple
        print(f"Episode {episode + 1} started")
        print("Initial observation shape:", obs.shape)
        print("Initial observation sample (first 10 points):", obs[:10])
        
        total_reward = 0
        for step in range(max_steps):
            action = env.action_space.sample()  # Random policy (replace with RL if needed)
            obs, reward, terminated, truncated, info = env.step(action)  # Unpack step output
            total_reward += reward
            
            # Render the environment to visualize training
            env.render()  # Displays 3D view in a window
            
            print(f"Step {step + 1}: Action: {action}, Reward: {reward}, Total Reward: {total_reward}")
            
            if terminated or truncated:
                print(f"Episode {episode + 1} ended early: Terminated={terminated}, Truncated={truncated}")
                break
        
        print(f"Episode {episode + 1} completed. Total Reward: {total_reward}")
    
    # Clean up
    env.close()
    print("Environment closed")

Exception ignored in: <function BaseGhostBodyNode.__del__ at 0x13bac5580>
Traceback (most recent call last):
  File "/Users/anikk/Downloads/RL_project/metadrive/metadrive/engine/physics_node.py", line 52, in __del__
    def __del__(self):

KeyboardInterrupt: 
[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), main_camera: MainCamera(1200, 900), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Since your screen is too small (1470, 956), we resize the window to (1147, 860). (engine_core.py:234)
2025-04-21 21:08:33.854 Python[88504:22320616] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-21 21:08:33.854 Python[88504:22320616] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[INFO] Start Scenario Index: 42, Num Scenarios : 1


Raw observation type: <class 'numpy.ndarray'>, shape: (259,)
Raw observation type: <class 'numpy.ndarray'>, shape: (259,)
Raw observation type: <class 'numpy.ndarray'>, shape: (259,)


In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
from stable_baselines3.common.env_checker import check_env
from metadrive.envs import MetaDriveEnv
from metadrive.engine.engine_utils import close_engine
import matplotlib.pyplot as plt
import imageio
import os

# --- SeedWrapper ---
class SeedWrapper(gym.Wrapper):
    def reset(self, seed=None, **kwargs):
        return self.env.reset(**kwargs)

# --- DiscreteActionWrapper ---
class DiscreteActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.discrete_actions = [
            np.array([0.0, 1.0, 0.0]),   # accelerate
            np.array([0.0, 0.0, 0.0]),   # no-op
            np.array([-0.5, 1.0, 0.0]),  # turn left while accelerating
            np.array([0.5, 1.0, 0.0]),   # turn right while accelerating
            np.array([0.0, 0.0, 1.0]),   # brake
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))
        
    def action(self, action_idx):
        action = self.discrete_actions[action_idx]
        print(f"Applying action {action_idx}: {action}")
        return action
    
    def reset(self, **kwargs):
        kwargs.pop("options", None)
        return self.env.reset(**kwargs)

# --- Custom Reward Wrapper ---
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.max_speed = 20.0
        self.lane_reward = 0.4  # Increased to encourage lane-keeping
        self.progress_reward = 0.8  # Increased to encourage movement
        self.brake_reward = 0.8  # Reduced to balance braking
        self.smooth_turn_reward = 0.3  # Increased for smoother driving
        self.collision_penalty = -1.5  # Increased penalty
        self.off_road_penalty = -1.0  # Increased penalty
        self.speed_penalty = -0.5  # Increased penalty
        self.intersection_speed_threshold = 0.4 * self.max_speed
        self.prev_steering = 0.0
        self.prev_velocity = 0.0
        
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        velocity = info.get("velocity", 0.0)
        cost = info.get("cost", 0.0)
        steering = self.env.discrete_actions[action][0]
        
        lidar_data = obs[:240]
        min_distance = np.min(lidar_data) if len(lidar_data) > 0 else 1.0
        
        out_of_road = info.get("out_of_road", False)
        near_intersection = min_distance < 0.4

        lane_bonus = self.lane_reward if not out_of_road else 0.0
        progress = (velocity / self.max_speed) * self.progress_reward
        
        brake_bonus = 0.0
        if near_intersection:
            if action == 4:
                if self.prev_velocity == 0.0:
                    brake_bonus = 0.4  # Reduced for stopped braking
                else:
                    brake_bonus = self.brake_reward
                print(f"Intersection Brake Rewarded: Velocity={velocity:.2f}, Min Distance={min_distance:.2f}")
            elif velocity < self.intersection_speed_threshold and velocity > 0.0:
                brake_bonus = self.brake_reward * 0.7
                print(f"Intersection Slow Speed Rewarded: Velocity={velocity:.2f}")
        
        steering_diff = abs(steering - self.prev_steering)
        smooth_turn = self.smooth_turn_reward if steering_diff < 0.3 else 0.0
        self.prev_steering = steering
        self.prev_velocity = velocity
        
        collision_cost = self.collision_penalty if cost > 0 else 0.0
        off_road_cost = self.off_road_penalty if out_of_road else 0.0
        speed_cost = self.speed_penalty if (velocity > self.intersection_speed_threshold and near_intersection) else 0.0
        
        custom_reward = (lane_bonus + progress + brake_bonus + smooth_turn + 
                        collision_cost + off_road_cost + speed_cost)
        custom_reward = np.clip(custom_reward, -2.0, 2.0)  # Adjusted clipping range
        
        if near_intersection:
            print(f"Intersection detected: Velocity={velocity:.2f}, Min Distance={min_distance:.2f}")
        if out_of_road:
            print(f"Off-road detected: Penalty applied")
        if velocity > 0.1:
            print(f"Moving: Velocity={velocity:.2f}")
        
        return obs, custom_reward, terminated, truncated, info

# --- Dueling DQN Network ---
class DuelingDQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DuelingDQN, self).__init__()
        self.feature = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.value = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        self.advantage = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )
    
    def forward(self, x):
        features = self.feature(x)
        value = self.value(features)
        advantage = self.advantage(features)
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

# --- Replay Buffer ---
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return (np.array(states), np.array(actions), np.array(rewards),
                np.array(next_states), np.array(dones))
    
    def __len__(self):
        return len(self.buffer)

# --- Dueling DQN Agent ---
class DuelingDQNAgent:
    def __init__(self, input_dim, output_dim, lr=0.0003, gamma=0.98, epsilon=1.0, epsilon_min=0.01, epsilon_decay_steps=2000, buffer_size=20000, batch_size=64):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = DuelingDQN(input_dim, output_dim).to(self.device)
        self.target_net = DuelingDQN(input_dim, output_dim).to(self.device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay_steps = epsilon_decay_steps
        self.buffer = ReplayBuffer(buffer_size)
        self.batch_size = batch_size
        self.output_dim = output_dim
        self.step_count = 0
        self.consecutive_brakes = 0
        self.max_consecutive_brakes = 3
        self.epsilon_history = []
    
    def select_action(self, state, greedy=False):
        self.step_count += 1
        if greedy:
            epsilon = 0.0
        else:
            epsilon = max(self.epsilon_min, self.epsilon - (self.epsilon - self.epsilon_min) * min(self.step_count, self.epsilon_decay_steps) / self.epsilon_decay_steps)
        self.epsilon_history.append(epsilon)
        
        if self.consecutive_brakes >= self.max_consecutive_brakes:
            action = random.choice([0, 1, 2, 3])
            self.consecutive_brakes = 0
        elif random.random() < epsilon:
            action = random.randrange(self.output_dim)
        else:
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.policy_net(state)
                print(f"Q-values: {q_values.cpu().numpy()[0]}")
            action = q_values.argmax().item()
        
        if action == 4:
            self.consecutive_brakes += 1
        else:
            self.consecutive_brakes = 0
        
        return action, epsilon  # Return epsilon for logging
    
    def train(self):
        if len(self.buffer) < self.batch_size:
            return
        states, actions, rewards, next_states, dones = self.buffer.sample(self.batch_size)
        
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        print("Training - States shape:", states.shape)
        
        q_values = self.policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        with torch.no_grad():
            next_q_values = self.target_net(next_states).max(1)[0]
            targets = rewards + self.gamma * next_q_values * (1 - dones)
        
        loss = nn.MSELoss()(q_values, targets)
        print(f"Training - Loss: {loss.item():.4f}")
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def update_target(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def save_model(self, filepath="dueling_dqn_model.pth"):
        torch.save(self.policy_net.state_dict(), filepath)
        print(f"Model saved to {filepath}")

# --- Main Training Loop ---
if __name__ == "__main__":
    close_engine()
    
    # Training config (human rendering)
    train_config = {
        "use_render": True,
        "manual_control": False,
        "traffic_density": 0.1,
        "num_scenarios": 1,
        "start_seed": 42,
        "vehicle_config": {
            "lidar": {"num_lasers": 240, "distance": 50.0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0}
        },
        "image_observation": False
    }
    
    # GIF config (top-down view for rgb_array)
    gif_config = {
        "use_render": False,
        "manual_control": False,
        "traffic_density": 0.1,
        "num_scenarios": 1,
        "start_seed": 42,
        "vehicle_config": {
            "lidar": {"num_lasers": 240, "distance": 50.0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0}
        },
        "image_observation": False,
        "top_down_camera_initial_x": 0,
        "top_down_camera_initial_y": 0,
        "top_down_camera_initial_z": 50
    }
    
    # Training environment
    env = MetaDriveEnv(train_config)
    env = SeedWrapper(env)
    env = DiscreteActionWrapper(env)
    env = CustomRewardWrapper(env)
    
    check_env(env)
    print("Observation space:", env.observation_space)
    print("Action space:", env.action_space)
    
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n
    agent = DuelingDQNAgent(input_dim, output_dim)
    
    num_episodes = 10  # Reduced for testing
    max_steps = 200
    target_update_freq = 5  # More frequent updates
    rewards_per_episode = []
    epsilon_history = []
    
    for episode in range(num_episodes):
        obs, info = env.reset()
        total_reward = 0
        
        print(f"Episode {episode + 1} started")
        print("Initial observation shape:", obs.shape)
        
        for step in range(max_steps):
            action, epsilon = agent.select_action(obs)
            epsilon_history.append(epsilon)
            next_obs, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            
            if action == 4:
                print(f"Step {step + 1}: Brake action applied! Velocity: {info.get('velocity', 0.0):.2f}")
            
            done = terminated or truncated
            agent.buffer.push(obs, action, reward, next_obs, done)
            
            agent.train()
            env.render()
            
            print(f"Step {step + 1}: Action: {action}, Reward: {reward:.4f}, Total Reward: {total_reward:.4f}, Epsilon: {epsilon:.3f}")
            
            obs = next_obs
            if done:
                print(f"Episode {episode + 1} ended early: Terminated={terminated}, Truncated={truncated}")
                break
        
        rewards_per_episode.append(total_reward)
        print(f"Episode {episode + 1} completed. Total Reward: {total_reward:.4f}")
        
        if episode % target_update_freq == 0:
            agent.update_target()
            print("Target network updated")
    
    # Save the model
    agent.save_model("dueling_dqn_model.pth")
    
    # Plot rewards and epsilon
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_episodes + 1), rewards_per_episode, label="Total Reward")
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title("Reward per Episode")
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(epsilon_history, label="Epsilon")
    plt.xlabel("Step")
    plt.ylabel("Epsilon")
    plt.title("Epsilon Decay")
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig("training_plots.png")
    plt.show()
    print("Plots saved as 'training_plots.png'")
    
    # Close training environment
    env.close()
    close_engine()
    
    # GIF creation environment
    env = MetaDriveEnv(gif_config)
    env = SeedWrapper(env)
    env = DiscreteActionWrapper(env)
    env = CustomRewardWrapper(env)
    
    obs, info = env.reset()
    total_reward = 0
    frames = []
    print("Creating GIF for greedy episode...")
    
    for step in range(max_steps):
        action, _ = agent.select_action(obs, greedy=True)
        next_obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        
        rgb_array = env.render()
        if rgb_array is not None:
            frames.append(rgb_array)
        else:
            print(f"Warning: Step {step + 1} - Render returned None, frame skipped.")
        
        print(f"Greedy Step {step + 1}: Action: {action}, Reward: {reward:.4f}, Total Reward: {total_reward:.4f}")
        
        obs = next_obs
        if terminated or truncated:
            print(f"Greedy episode ended: Terminated={terminated}, Truncated={truncated}")
            break
    
    # Save frames as GIF
    if frames:
        gif_path = "greedy_episode.gif"
        imageio.mimsave(gif_path, frames, fps=30)
        print(f"GIF saved as '{gif_path}' with {len(frames)} frames")
    else:
        print("No frames captured, GIF not created.")
    
    print(f"Greedy episode completed. Total Reward: {total_reward:.4f}")
    env.close()
    print("Environment closed")

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), main_camera: MainCamera(1200, 900), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Since your screen is too small (1470, 956), we resize the window to (1147, 860). (engine_core.py:234)
2025-04-20 17:01:04.765 Python[88796:20745527] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-20 17:01:04.765 Python[88796:20745527] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[INFO] Start Scenario Index: 42, Num Scenarios : 1


Applying action 3: [0.5 1.  0. ]
Intersection Slow Speed Rewarded: Velocity=0.28
Intersection detected: Velocity=0.28, Min Distance=0.00
Moving: Velocity=0.28
Applying action 1: [0. 0. 0.]
Intersection Slow Speed Rewarded: Velocity=0.03
Intersection detected: Velocity=0.03, Min Distance=0.00
Applying action 2: [-0.5  1.   0. ]
Intersection Slow Speed Rewarded: Velocity=0.31
Intersection detected: Velocity=0.31, Min Distance=0.00
Moving: Velocity=0.31
Applying action 4: [0. 0. 1.]
Intersection Brake Rewarded: Velocity=0.27, Min Distance=0.00
Intersection detected: Velocity=0.27, Min Distance=0.00
Moving: Velocity=0.27
Applying action 2: [-0.5  1.   0. ]
Intersection Slow Speed Rewarded: Velocity=0.54
Intersection detected: Velocity=0.54, Min Distance=0.00
Moving: Velocity=0.54
Applying action 2: [-0.5  1.   0. ]
Intersection Slow Speed Rewarded: Velocity=0.81
Intersection detected: Velocity=0.81, Min Distance=0.00
Moving: Velocity=0.81
Applying action 0: [0. 1. 0.]
Intersection Slow Spe

SystemExit: 

/Users/anikk/Downloads/RL_project/metadrive-env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import gymnasium as gym
import torch
import numpy as np
import random
from collections import deque
from torch import nn, optim
from metadrive.envs import MetaDriveEnv
from metadrive.engine.engine_utils import close_engine

# --- SeedWrapper ---
class SeedWrapper(gym.Wrapper):
    def reset(self, seed=None, **kwargs):
        return self.env.reset(**kwargs)

# --- DiscreteActionWrapper ---
class DiscreteActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.discrete_actions = [
            np.array([0.0, 1.0, 0.0]),   # accelerate
            np.array([0.0, 0.0, 0.0]),   # no-op
            np.array([-0.5, 1.0, 0.0]),  # turn left
            np.array([0.5, 1.0, 0.0]),   # turn right
            np.array([0.0, 0.0, 1.0])    # brake
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))

    def action(self, action_idx):
        return self.discrete_actions[action_idx]

    def reset(self, **kwargs):
        kwargs.pop("options", None)
        return self.env.reset(**kwargs)

# --- LidarOnlyObservationWrapper ---
class LidarOnlyObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, lidar_points=240):
        super().__init__(env)
        self.lidar_points = lidar_points
        self.observation_space = gym.spaces.Box(low=0.0, high=50.0, shape=(lidar_points,), dtype=np.float32)

    def observation(self, obs):
        if not isinstance(obs, np.ndarray):
            obs = np.array(obs)
        return np.clip(obs[:self.lidar_points], 0.0, 50.0).astype(np.float32)

# --- Replay Buffer ---
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.stack(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.stack(next_states),
            np.array(dones, dtype=np.float32)
        )

    def __len__(self):
        return len(self.buffer)

# --- DQN Network ---
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# --- Epsilon-Greedy Action Selection ---
def select_action(state, epsilon, policy_net, action_dim):
    if np.random.rand() < epsilon:
        return random.randint(0, action_dim - 1)
    with torch.no_grad():
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        return policy_net(state_tensor).argmax().item()

# --- MAIN ---
if __name__ == "__main__":
    close_engine()

    # MetaDrive Configuration
    config = {
        "use_render": True,
        "manual_control": False,
        "traffic_density": 0.05,
        "num_scenarios": 1,
        "start_seed": 42,
        "vehicle_config": {
            "lidar": {"num_lasers": 240, "distance": 50.0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0}
        },
        "image_observation": False
    }

    # Create and wrap environment
    env = MetaDriveEnv(config)
    env = SeedWrapper(env)
    env = DiscreteActionWrapper(env)
    env = LidarOnlyObservationWrapper(env, lidar_points=240)

    obs_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    # Initialize networks, optimizer, replay buffer
    policy_net = DQN(obs_dim, action_dim)
    target_net = DQN(obs_dim, action_dim)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-3)
    buffer = ReplayBuffer(capacity=10000)

    # Hyperparameters
    gamma = 0.99
    batch_size = 64
    epsilon_start = 1.0
    epsilon_end = 0.05
    epsilon_decay = 500
    target_update_freq = 100
    num_episodes = 500
    max_steps = 300
    steps_done = 0

    for episode in range(num_episodes):
        obs, _ = env.reset()
        state = obs
        total_reward = 0

        for step in range(max_steps):
            epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * steps_done / epsilon_decay)
            action = select_action(state, epsilon, policy_net, action_dim)
            obs, reward, terminated, truncated, _ = env.step(action)
            next_state = obs
            done = terminated or truncated

            buffer.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            steps_done += 1

            if len(buffer) >= batch_size:
                s_batch, a_batch, r_batch, ns_batch, d_batch = buffer.sample(batch_size)

                s_batch = torch.FloatTensor(s_batch)
                a_batch = torch.LongTensor(a_batch).unsqueeze(1)
                r_batch = torch.FloatTensor(r_batch).unsqueeze(1)
                ns_batch = torch.FloatTensor(ns_batch)
                d_batch = torch.FloatTensor(d_batch).unsqueeze(1)

                q_values = policy_net(s_batch).gather(1, a_batch)
                next_q = target_net(ns_batch).max(1)[0].unsqueeze(1)
                expected_q = r_batch + gamma * next_q * (1 - d_batch)

                loss = nn.MSELoss()(q_values, expected_q)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if steps_done % target_update_freq == 0:
                target_net.load_state_dict(policy_net.state_dict())

            if done:
                print(f"[Episode {episode + 1}] Steps: {step + 1}, Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.3f}")
                break

        env.render()

    env.close()
    print("Training finished and environment closed.")


[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), main_camera: MainCamera(1200, 900), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Since your screen is too small (1470, 956), we resize the window to (1147, 860). (engine_core.py:234)
[INFO] Start Scenario Index: 42, Num Scenarios : 1


[Episode 1] Steps: 155, Total Reward: 93.62, Epsilon: 0.748
[Episode 2] Steps: 73, Total Reward: 21.76, Epsilon: 0.653
[Episode 3] Steps: 132, Total Reward: 94.48, Epsilon: 0.513
[Episode 4] Steps: 54, Total Reward: 8.70, Epsilon: 0.466


SystemExit: 

: 

## DDQN

In [ ]:
import gymnasium as gym
import torch
import numpy as np
import random
import os
import time
from collections import deque
from torch import nn, optim
import torch.nn.functional as F
from metadrive.envs import MetaDriveEnv
from metadrive.engine.engine_utils import close_engine
from metadrive.component.sensors.lidar import Lidar
from metadrive.utils import clip  # Added import for clip function

class SeedWrapper(gym.Wrapper):
    def __init__(self, env, fixed_seed=42):
        super().__init__(env)
        self.fixed_seed = fixed_seed
        # Set the random seed for numpy and random to ensure consistent traffic
        np.random.seed(self.fixed_seed)
        random.seed(self.fixed_seed)

    def reset(self, seed=None, options=None, **kwargs):
        # Always use the fixed seed for consistent map and traffic
        kwargs['seed'] = self.fixed_seed
        return self.env.reset(**kwargs)

class DiscreteActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.discrete_actions = [
            np.array([0.0, 1.0, 0.0]), # accelerate
            np.array([0.0, 0.0, 0.0]), # no-op
            np.array([-0.5, 1.0, 0.0]), # turn left
            np.array([0.5, 1.0, 0.0]), # turn right
            np.array([0.0, 0.0, 1.0]), # brake
            np.array([-0.2, 0.5, 0.0]), # soft left (finer steering)
            np.array([0.2, 0.5, 0.0]), # soft right (finer steering)
            np.array([0.0, 0.5, 0.0]), # gentle accelerate
            np.array([-0.05, 0.8, 0.0]), # very slight left (for lane keeping)
            np.array([0.05, 0.8, 0.0]), # very slight right (for lane keeping)
            np.array([-0.3, 0.8, 0.0]), # medium left
            np.array([0.3, 0.8, 0.0]), # medium right
            np.array([-0.2, 0.3, 0.0]), # slow left turn
            np.array([0.2, 0.3, 0.0]), # slow right turn
            np.array([-0.1, 1.0, 0.0]), # gentle left at full speed
            np.array([0.1, 1.0, 0.0]), # gentle right at full speed
            np.array([-0.025, 0.8, 0.0]), # micro left (for precise lane keeping)
            np.array([0.025, 0.8, 0.0]) # micro right (for precise lane keeping)
        ]
        self.action_space = gym.spaces.Discrete(len(self.discrete_actions))

    def action(self, action_idx):
        return self.discrete_actions[action_idx]

    def reset(self, **kwargs):
        kwargs.pop("options", None)
        return self.env.reset(**kwargs)

class EnhancedObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, lidar_points=240):
        super().__init__(env)
        self.lidar_points = lidar_points
        # Extended observation space to include intersection-related features
        self.observation_space = gym.spaces.Box(
            low=-float('inf'), 
            high=float('inf'), 
            shape=(lidar_points + 10,), # Added 2 more features for intersection
            dtype=np.float32
        )

    def observation(self, obs):
        base_env = self.env.unwrapped
        vehicle = base_env.agents.get("agent0")
        lidar_obs = np.clip(obs[:self.lidar_points], 0.0, 50.0).astype(np.float32)
        lane_features = np.zeros(10, dtype=np.float32)
        
        if vehicle and vehicle.lane and vehicle.navigation:
            try:
                if vehicle.lane in vehicle.navigation.current_ref_lanes:
                    current_lane = vehicle.lane
                else:
                    current_lane = vehicle.navigation.current_ref_lanes[0]
                
                _, lateral_dist = current_lane.local_coordinates(vehicle.position)
                lane_width = vehicle.navigation.get_current_lane_width()
                normalized_lateral = lateral_dist / (lane_width/2)
                lane_heading = current_lane.heading_at(vehicle.position)
                heading_diff = np.abs(np.mod(vehicle.heading_theta - lane_heading, 2 * np.pi))
                heading_diff = min(heading_diff, 2 * np.pi - heading_diff) / np.pi
                dist_to_intersection = 50.0
                if hasattr(vehicle.navigation, 'next_intersection_distance'):
                    dist_to_intersection = vehicle.navigation.next_intersection_distance()
                target_lane_index = vehicle.navigation.next_target_lane_index
                suggested_steering = 0.0
                if target_lane_index is not None and target_lane_index != vehicle.lane_index:
                    suggested_steering = 1.0 if target_lane_index > vehicle.lane_index else -1.0
                speed_norm = vehicle.speed_km_h / vehicle.max_speed_km_h
                
                # New intersection-related features
                intersection_approach = 1.0 - min(dist_to_intersection / 20.0, 1.0) # Proximity to intersection
                target_heading_diff = 0.0
                if vehicle.navigation.is_in_intersection() and hasattr(vehicle.navigation, 'next_target_lane'):
                    next_lane = vehicle.navigation.next_target_lane
                    if next_lane:
                        target_heading = next_lane.heading_at(next_lane.start)
                        target_heading_diff = np.abs(np.mod(vehicle.heading_theta - target_heading, 2 * np.pi))
                        target_heading_diff = min(target_heading_diff, 2 * np.pi - target_heading_diff) / np.pi
                
                lane_features[0] = normalized_lateral
                lane_features[1] = heading_diff
                lane_features[2] = min(1.0, dist_to_intersection / 50.0)
                lane_features[3] = suggested_steering
                lane_features[4] = speed_norm
                lane_features[5] = 1.0 if vehicle.navigation.is_in_intersection() else 0.0
                lane_features[6] = vehicle.steering / vehicle.max_steering
                lane_features[7] = 1.0 if vehicle.navigation.is_on_straight_road() else 0.0
                lane_features[8] = intersection_approach
                lane_features[9] = target_heading_diff
            except:
                pass
        
        combined_obs = np.concatenate([lidar_obs, lane_features])
        return combined_obs

class FrameStackWrapper(gym.Wrapper):
    def __init__(self, env, num_stack=3):
        super().__init__(env)
        self.num_stack = num_stack
        self.frames = deque(maxlen=num_stack)
        obs_shape = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=np.repeat(env.observation_space.low[np.newaxis, ...], num_stack, axis=0).flatten(),
            high=np.repeat(env.observation_space.high[np.newaxis, ...], num_stack, axis=0).flatten(),
            shape=(obs_shape[0] * num_stack,),
            dtype=env.observation_space.dtype
        )

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        for _ in range(self.num_stack):
            self.frames.append(obs)
        return self._get_observation(), info

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_observation(), reward, terminated, truncated, info

    def _get_observation(self):
        return np.concatenate(list(self.frames))

class ReplayBuffer:
    def __init__(self, capacity=100000, alpha=0.6, beta=0.4, beta_increment=1e-5):
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0
        self.size = 0
        self.alpha = alpha
        self.beta = beta
        self.beta_increment = beta_increment
        self.max_priority = 1.0

    def push(self, state, action, reward, next_state, done):
        max_priority = self.max_priority if self.size > 0 else 1.0
        if self.size < self.capacity:
            self.buffer.append((state, action, reward, next_state, done))
            self.priorities[self.position] = max_priority
            self.size += 1
        else:
            self.buffer[self.position] = (state, action, reward, next_state, done)
            self.priorities[self.position] = max_priority
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        self.beta = min(1.0, self.beta + self.beta_increment)
        if self.size < batch_size:
            batch = random.sample(list(self.buffer), self.size)
            indices = np.random.choice(self.size, self.size, replace=False)
        else:
            priorities = self.priorities[:self.size] ** self.alpha
            probabilities = priorities / np.sum(priorities)
            indices = np.random.choice(self.size, batch_size, replace=False, p=probabilities)
            batch = [self.buffer[i] for i in indices]
            weights = (self.size * probabilities[indices]) ** (-self.beta)
            weights /= np.max(weights)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.stack(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.stack(next_states),
            np.array(dones, dtype=np.float32),
            indices,
            np.array(weights, dtype=np.float32)
        )

    def update_priorities(self, indices, priorities):
        for idx, priority in zip(indices, priorities):
            self.priorities[idx] = priority
        self.max_priority = max(self.max_priority, priority)

    def __len__(self):
        return self.size

class EnhancedDuelingDDQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EnhancedDuelingDDQN, self).__init__()
        self.feature_layer = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU()
        )
        self.value_stream = nn.Sequential(
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        self.advantage_stream = nn.Sequential(
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        features = self.feature_layer(x)
        value = self.value_stream(features)
        advantage = self.advantage_stream(features)
        q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))
        return q_values

def select_action(state, epsilon, policy_net, action_dim, device):
    if np.random.rand() < epsilon:
        return random.randint(0, action_dim - 1)
    with torch.no_grad():
        policy_net.eval()
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        action = policy_net(state_tensor).argmax().item()
        policy_net.train()
    return action

def custom_reward_function(env, vehicle_id: str):
    vehicle = env.agents[vehicle_id]
    step_info = dict()
    reward = 0.0

    # Basic reward predefined
    if vehicle.lane in vehicle.navigation.current_ref_lanes:
        current_lane = vehicle.lane
        positive_road = 1
    else:
        current_lane = vehicle.navigation.current_ref_lanes[0]
        current_road = vehicle.navigation.current_road
        positive_road = 1 if not current_road.is_negative_road() else -1

    long_last, _ = current_lane.local_coordinates(vehicle.last_position)
    long_now, lateral_now = current_lane.local_coordinates(vehicle.position)
    lane_width = vehicle.navigation.get_current_lane_width()

    lateral_factor = clip(1 - 2 * abs(lateral_now) / lane_width, 0.0, 1.0) if env.config["use_lateral_reward"] else 1.0

    reward += env.config["driving_reward"] * (long_now - long_last) * lateral_factor * positive_road
    reward += env.config["speed_reward"] * (vehicle.speed_km_h / vehicle.max_speed_km_h) * positive_road

    # Extra reward for maintaining in the center of lane for stablity
    if abs(lateral_now) < 0.5:
        reward += 0.3

    # Penalty for high speed (Required in DQN for discrete action space)
    if vehicle.speed_km_h > 20:
        reward -= 1.0

    # Curve reward and penalty
    try:
        road_type = getattr(vehicle.navigation.current_road, "road_type", "").lower()
        is_intersection = any(key in road_type for key in ["intersection", "roundabout"])
    except Exception:
        is_intersection = False

    try:
        curvature = abs(current_lane.curvature_at(long_now))
        if curvature > 0.02:
            if vehicle.speed_km_h > 20:
                reward -= 0.5
            else:
                reward += 0.2
    except:
        curvature = 0.0

    if is_intersection:
        if vehicle.speed_km_h > 15:
            reward -= 1.0
        try:
            future_pos = vehicle.navigation.current_ref_lanes[0].position(long_now + 2.0, 0)
            expected_heading = current_lane.heading_at(long_now + 2.0)
            heading_diff = abs(vehicle.heading_theta - expected_heading)
            if heading_diff < 20:
                reward += 0.3
            else:
                reward -= 0.5
        except:
            reward -= 0.1

        try:
            final_long, _ = current_lane.local_coordinates(vehicle.navigation.checkpoints[-1])
            if abs(final_long - long_now) < 5:
                reward += 0.4
        except:
            pass

    # Steering smoothness
    if hasattr(vehicle, "steering_change"):
        steering_change = abs(vehicle.steering_change)
        if steering_change > 0.3:
            reward -= 0.2
        else:
            reward += 0.1

    # Add more penalty for crashing into sidewalk
    if vehicle.crash_sidewalk:
        reward -= 4.0

    # Avoid crashing with other vehicles (Not apply now for not using lidar data)
    detected_objects = vehicle.lidar.get_surrounding_objects(vehicle)
    front_cars = []
    side_lanes_clear = {"left": True, "right": True}

    for obj in detected_objects:
        if hasattr(obj, "position"):
            rel_pos = vehicle.convert_to_local_coordinates(obj.position, vehicle.position)
            rel_x, rel_y = rel_pos[0], rel_pos[1]
            dist = np.linalg.norm(rel_pos)
            if 0 < rel_x < 25 and abs(rel_y) < 2.0:
                front_cars.append((obj, dist))
            if abs(rel_y) > 2.0 and dist < 20:
                if rel_y > 0:
                    side_lanes_clear["left"] = False
                else:
                    side_lanes_clear["right"] = False

    front_cars.sort(key=lambda x: x[1])

    if front_cars:
        if side_lanes_clear["left"] or side_lanes_clear["right"]:
            reward += 0.2
        else:
            reward -= 0.5
            if vehicle.speed_km_h > 20:
                reward -= 0.5

    # Terminal state
    if env._is_arrive_destination(vehicle):
        reward = +env.config["success_reward"]
    elif env._is_out_of_road(vehicle):
        reward = -env.config["out_of_road_penalty"]
    elif vehicle.crash_vehicle:
        reward = -env.config["crash_vehicle_penalty"]
    elif vehicle.crash_object:
        reward = -env.config["crash_object_penalty"]
    elif vehicle.crash_sidewalk:
        reward = -env.config["crash_sidewalk_penalty"]

    step_info["step_reward"] = reward
    step_info["route_completion"] = vehicle.navigation.route_completion
    return reward, step_info

class LinearLRScheduler:
    def __init__(self, optimizer, start_lr, end_lr, decay_steps):
        self.optimizer = optimizer
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.decay_steps = decay_steps
        self.current_step = 0
        
    def step(self):
        self.current_step += 1
        progress = min(1.0, self.current_step / self.decay_steps)
        lr = self.start_lr + progress * (self.end_lr - self.start_lr)
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        return lr

def train_ddqn(env, policy_net, target_net, optimizer, buffer, num_episodes, 
              batch_size, gamma, epsilon_start, epsilon_end, epsilon_decay, 
              target_update_freq, lr_scheduler, device="cpu"):
    
    episode_rewards = []
    episode_lengths = []
    evaluation_scores = []
    
    os.makedirs("models", exist_ok=True)
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        episode_length = 0
        done = False
        epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-episode / epsilon_decay)
        
        while not done:
            action = select_action(state, epsilon, policy_net, env.action_space.n, device)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            vehicle = env.unwrapped.agents.get("agent0")
            if vehicle:
                # Using the new custom reward function
                reward, _ = custom_reward_function(env.unwrapped, "agent0")
            
            buffer.push(state, action, reward, next_state, done)
            state = next_state
            episode_reward += reward
            episode_length += 1
            
            if len(buffer) > batch_size:
                states, actions, rewards, next_states, dones, indices, weights = buffer.sample(batch_size)
                
                states_tensor = torch.FloatTensor(states).to(device)
                actions_tensor = torch.LongTensor(actions).to(device)
                rewards_tensor = torch.FloatTensor(rewards).to(device)
                next_states_tensor = torch.FloatTensor(next_states).to(device)
                dones_tensor = torch.FloatTensor(dones).to(device)
                weights_tensor = torch.FloatTensor(weights).to(device)
                
                current_q_values = policy_net(states_tensor).gather(1, actions_tensor.unsqueeze(1)).squeeze(1)
                
                with torch.no_grad():
                    next_action = policy_net(next_states_tensor).argmax(dim=1)
                    next_q_values = target_net(next_states_tensor).gather(1, next_action.unsqueeze(1)).squeeze(1)
                    target_q_values = rewards_tensor + gamma * next_q_values * (1 - dones_tensor)
                
                loss = (weights_tensor * F.smooth_l1_loss(current_q_values, target_q_values, reduction='none')).mean()
                
                td_errors = torch.abs(current_q_values - target_q_values).detach().cpu().numpy()
                buffer.update_priorities(indices, td_errors + 1e-5)
                
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0)
                optimizer.step()
            
            lr_scheduler.step()
            
            if episode_length % target_update_freq == 0:
                target_net.load_state_dict(policy_net.state_dict())
        
        episode_rewards.append(episode_reward)
        episode_lengths.append(episode_length)
        
        if episode % 100 == 0:
            torch.save(policy_net.state_dict(), f"models/ddqn_episode_{episode}.pth")
        
        if episode % 50 == 0:
            eval_reward = evaluate_policy(env, policy_net, device)
            evaluation_scores.append(eval_reward)
            print(f"Episode {episode}/{num_episodes}, Reward: {episode_reward:.2f}, "
                 f"Length: {episode_length}, Epsilon: {epsilon:.4f}, Eval Reward: {eval_reward:.2f}")
        
        if len(evaluation_scores) > 20 and np.mean(evaluation_scores[-10:]) > 200:
            print("Early stopping triggered: consistent high performance achieved")
            break
    
    return episode_rewards, episode_lengths, evaluation_scores

def evaluate_policy(env, policy_net, device, num_episodes=5):
    policy_net.eval()
    total_reward = 0
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action = select_action(state, 0.0, policy_net, env.action_space.n, device)
            next_state, reward, terminated, truncated, info = env.step(action)
            vehicle = env.unwrapped.agents.get("agent0")
            if vehicle:
                # Using the new custom reward function
                reward, _ = custom_reward_function(env.unwrapped, "agent0")
            episode_reward += reward
            state = next_state
            done = terminated or truncated
        total_reward += episode_reward
    policy_net.train()
    return total_reward / num_episodes

if __name__ == "__main__":
    config = {
        "num_scenarios": 1, # Single map
        "use_render": True,
        "manual_control": False,
        "traffic_density": 0.1,
        "start_seed": 42, # Fixed seed for consistent map
        "driving_reward": 1.0,
        "speed_reward": 0.5,
        "success_reward": 50.0,
        "out_of_road_penalty": 50.0,
        "crash_vehicle_penalty": 50.0,
        "crash_object_penalty": 50.0,
        "crash_sidewalk_penalty": 50.0,  # Added penalty for sidewalk crashes
        "use_lateral_reward": True,
        "random_traffic": False, # Consistent traffic pattern
        "vehicle_config": {
            "lidar": {"num_lasers": 240, "distance": 50.0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0}
        },
        "image_observation": False
    }
    close_engine()
    env = MetaDriveEnv(config)
    env = SeedWrapper(env, fixed_seed=42) # Enforce fixed seed for map and traffic
    env = EnhancedObservationWrapper(env, lidar_points=240)
    env = DiscreteActionWrapper(env)
    env = FrameStackWrapper(env, num_stack=3)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n
    policy_net = EnhancedDuelingDDQN(input_dim, output_dim).to(device)
    target_net = EnhancedDuelingDDQN(input_dim, output_dim).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)
    lr_scheduler = LinearLRScheduler(optimizer, start_lr=1e-4, end_lr=1e-5, decay_steps=100000)

    buffer = ReplayBuffer(capacity=100000)

    num_episodes = 1000
    batch_size = 64
    gamma = 0.99
    epsilon_start = 1.0
    epsilon_end = 0.05
    epsilon_decay = 200
    target_update_freq = 1000

    try:
        rewards, lengths, eval_scores = train_ddqn(
            env, policy_net, target_net, optimizer, buffer, num_episodes,
            batch_size, gamma, epsilon_start, epsilon_end, epsilon_decay,
            target_update_freq, lr_scheduler, device
        )
    finally:
        env.close()
        close_engine()

    torch.save(policy_net.state_dict(), "models/ddqn_final.pth")


[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), main_camera: MainCamera(1200, 900), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
[WARNING] Since your screen is too small (1470, 956), we resize the window to (1147, 860). (engine_core.py:234)
2025-04-21 20:15:31.974 Python[78283:22260908] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-21 20:15:31.974 Python[78283:22260908] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[INFO] Start Scenario Index: 42, Num Scenarios : 1


SystemExit: 

/Users/anikk/Downloads/RL_project/metadrive-env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## RGB dataset

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
from metadrive.envs import MetaDriveEnv
from metadrive.component.sensors.rgb_camera import RGBCamera
from metadrive.engine.engine_utils import close_engine

# --- Replay Buffer ---
class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            torch.stack(states),
            torch.tensor(actions, dtype=torch.long),
            torch.tensor(rewards, dtype=torch.float32),
            torch.stack(next_states),
            torch.tensor(dones, dtype=torch.float32)
        )

    def __len__(self):
        return len(self.buffer)

# --- CNN Q-network ---
class CNN_QNetwork(nn.Module):
    def __init__(self, input_channels, num_actions, input_height=84, input_width=84):
        super(CNN_QNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        # Calculate feature map size dynamically
        def conv2d_size_out(size, kernel_size, stride):
            return (size - kernel_size) // stride + 1
        
        h = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_height, 8, 4), 4, 2), 3, 1)
        w = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_width, 8, 4), 4, 2), 3, 1)
        linear_input_size = 64 * h * w
        
        self.fc = nn.Sequential(
            nn.Linear(linear_input_size, 512),
            nn.ReLU(),
            nn.Linear(512, num_actions)
        )

    def forward(self, x):
        x = x / 255.0  # Normalize
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# --- Action Selection ---
def select_action(state, epsilon, policy_net, num_actions):
    if random.random() < epsilon:
        return random.randint(0, num_actions - 1)
    with torch.no_grad():
        q_values = policy_net(state.unsqueeze(0))
        return q_values.argmax().item()

# --- Preprocessing ---
def preprocess_observation(obs):
    # Handle different observation formats
    if isinstance(obs, dict) and 'rgb' in obs:
        obs = obs['rgb']
    
    # Convert to tensor with correct dimension ordering
    if isinstance(obs, np.ndarray):
        if len(obs.shape) == 3 and obs.shape[-1] == 3:  # (H, W, C) format
            obs = torch.tensor(obs, dtype=torch.float32).permute(2, 0, 1)
        elif len(obs.shape) == 3 and obs.shape[0] == 3:  # (C, H, W) format
            obs = torch.tensor(obs, dtype=torch.float32)
        else:
            raise ValueError(f"Unexpected observation shape: {obs.shape}")
    return obs

# --- Training Loop ---
def train():
    close_engine()

    width, height = 84, 84
    config = {
        "use_render": False,  # Enable rendering
        "manual_control": False,
        "image_observation": True,
        "norm_pixel": False,
        "vehicle_config": {
            "lidar": {"num_lasers": 0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0},
            "image_source": "rgb"  # Add this to specify which camera to use
        },
        "sensors": {
            "rgb": (RGBCamera, width, height)
        },
        "traffic_density": 0.05,
        "num_scenarios": 1,
        "start_seed": 42,
        "window_size": (1200, 800),  # Set window size for rendering
        "show_interface": True,      # Show driving interface
        "show_fps": True             # Show FPS counter
    }

    env = MetaDriveEnv(config)
    obs, _ = env.reset()
    
    # Debug observation format
    print(f"Observation type: {type(obs)}")
    if isinstance(obs, dict):
        print(f"Observation keys: {obs.keys()}")
        if 'rgb' in obs:
            print(f"RGB shape: {obs['rgb'].shape}")
    elif isinstance(obs, np.ndarray):
        print(f"Observation shape: {obs.shape}")
    
    try:
        state = preprocess_observation(obs)
        print(f"Processed state shape: {state.shape}")
        input_channels = state.shape[0]  # Usually 3 for RGB
    except Exception as e:
        print(f"Error processing initial observation: {e}")
        # Fallback to assuming RGB (3 channels)
        input_channels = 3
        
    num_actions = 5

    policy_net = CNN_QNetwork(input_channels, num_actions, height, width)
    target_net = CNN_QNetwork(input_channels, num_actions, height, width)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-4)
    buffer = ReplayBuffer(capacity=10000)

    num_episodes = 500
    max_steps = 300
    batch_size = 32
    gamma = 0.99
    epsilon_start = 1.0
    epsilon_end = 0.05
    epsilon_decay = 10000
    target_update_freq = 1000
    steps_done = 0

    for episode in range(num_episodes):
        obs, _ = env.reset()
        try:
            state = preprocess_observation(obs)
        except Exception as e:
            print(f"Error processing observation at episode {episode}: {e}")
            continue
            
        total_reward = 0

        for step in range(max_steps):
            # Render the environment (3D view)
            env.render()
            
            epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * steps_done / epsilon_decay)
            
            try:
                action = select_action(state, epsilon, policy_net, num_actions)
            except Exception as e:
                print(f"Error selecting action: {e}")
                action = random.randint(0, num_actions - 1)

            # Discrete → Continuous action
            act = {
                0: np.array([0.0, 1.0, 0.0]),  # Forward
                1: np.array([0.0, 0.0, 0.0]),  # No action
                2: np.array([-0.5, 1.0, 0.0]), # Left turn
                3: np.array([0.5, 1.0, 0.0]),  # Right turn
                4: np.array([0.0, 0.0, 1.0])   # Brake
            }[action]

            obs, reward, terminated, truncated, _ = env.step(act)
            
            try:
                next_state = preprocess_observation(obs)
            except Exception as e:
                print(f"Error processing next observation: {e}")
                continue
                
            done = terminated or truncated

            buffer.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            steps_done += 1

            # Training
            if len(buffer) >= batch_size:
                try:
                    s_batch, a_batch, r_batch, ns_batch, d_batch = buffer.sample(batch_size)

                    q_values = policy_net(s_batch).gather(1, a_batch.unsqueeze(1)).squeeze(1)
                    next_actions = policy_net(ns_batch).argmax(1)
                    next_q = target_net(ns_batch).gather(1, next_actions.unsqueeze(1)).squeeze(1)
                    expected_q = r_batch + gamma * next_q * (1 - d_batch)

                    loss = nn.MSELoss()(q_values, expected_q.detach())
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                except Exception as e:
                    print(f"Error during training: {e}")

            if steps_done % target_update_freq == 0:
                target_net.load_state_dict(policy_net.state_dict())

            if done:
                break

        print(f"Episode {episode + 1}: Total Reward: {total_reward:.2f}, Epsilon: {epsilon:.3f}")

    env.close()

if __name__ == "__main__":
    train()


[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), rgb: RGBCamera(84, 84)]
[INFO] Render Mode: offscreen
[INFO] Horizon (Max steps per agent): 1000
[WARNING] You have set norm_pixel = False, which means the observation will be uint8 values in [0, 255]. Please make sure you have parsed them later before feeding them to network! (metadrive_env.py:117)
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe


In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from collections import namedtuple
from metadrive.envs import MetaDriveEnv
from metadrive.component.sensors.rgb_camera import RGBCamera
from metadrive.engine.engine_utils import close_engine

# --- PPO Configuration ---
class PPOConfig:
    def __init__(self):
        self.gamma = 0.99  # Discount factor
        self.lam = 0.95   # GAE lambda
        self.clip_param = 0.2  # PPO clipping parameter
        self.value_loss_coef = 0.5
        self.entropy_coef = 0.01
        self.lr = 3e-4
        self.eps = 1e-5
        self.max_grad_norm = 0.5
        self.ppo_epochs = 10
        self.batch_size = 64
        self.num_episodes = 500
        self.max_steps = 300
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- CNN Actor-Critic Network ---
class CNN_ActorCritic(nn.Module):
    def __init__(self, input_channels, num_actions, input_height=84, input_width=84):
        super(CNN_ActorCritic, self).__init__()
        # Shared convolutional backbone
        self.conv = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        # Calculate feature map size
        def conv2d_size_out(size, kernel_size, stride):
            return (size - kernel_size) // stride + 1
        
        # Fixed syntax for size calculation
        h = conv2d_size_out(
            conv2d_size_out(
                conv2d_size_out(input_height, 8, 4),
                4, 2
            ),
            3, 1
        )
        w = conv2d_size_out(
            conv2d_size_out(
                conv2d_size_out(input_width, 8, 4),
                4, 2
            ),
            3, 1
        )
        linear_input_size = 64 * h * w
        
        # Actor head
        self.actor = nn.Sequential(
            nn.Linear(linear_input_size, 512),
            nn.ReLU(),
            nn.Linear(512, num_actions),
            nn.Softmax(dim=-1)
        )
        
        # Critic head
        self.critic = nn.Sequential(
            nn.Linear(linear_input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        x = x / 255.0
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        action_probs = self.actor(x)
        value = self.critic(x)
        return action_probs, value

    def act(self, state):
        state = state.unsqueeze(0).to(config.device)
        with torch.no_grad():
            probs, value = self.forward(state)
            dist = torch.distributions.Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)
        return action.item(), log_prob.item(), value.item()

# --- Preprocessing ---
def preprocess_observation(obs):
    # Handle dictionary observation
    if isinstance(obs, dict):
        if 'image' not in obs:
            raise ValueError(f"Observation dictionary does not contain 'image' key: {obs.keys()}")
        obs = obs['image']
    elif isinstance(obs, np.ndarray):
        pass
    else:
        raise ValueError(f"Unexpected observation type: {type(obs)}")
    
    # Handle 4D observation by selecting first element in first dimension
    if len(obs.shape) == 4:
        obs = obs[0]  # Assume batch dimension and take first element
    
    # Validate and convert to tensor
    if len(obs.shape) == 3 and obs.shape[-1] == 3:  # (H, W, C)
        obs = torch.tensor(obs, dtype=torch.float32).permute(2, 0, 1)  # To (C, H, W)
    elif len(obs.shape) == 3 and obs.shape[0] == 3:  # (C, H, W)
        obs = torch.tensor(obs, dtype=torch.float32)
    else:
        raise ValueError(f"Unexpected observation shape: {obs.shape}")
    
    return obs.to(config.device)

# --- Generalized Advantage Estimation ---
def compute_gae(rewards, values, next_value, dones, gamma, lam):
    advantages = []
    gae = 0
    next_value_t = next_value
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * next_value_t * (1 - dones[i]) - values[i]
        gae = delta + gamma * lam * (1 - dones[i]) * gae
        advantages.insert(0, gae)
        next_value_t = values[i]
    return advantages

# --- PPO Training Loop ---
def train_ppo():
    close_engine()
    
    # Environment configuration
    width, height = 84, 84
    env_config = {
        "use_render": True,
        "manual_control": False,
        "image_observation": True,
        "norm_pixel": False,
        "vehicle_config": {
            "lidar": {"num_lasers": 0},
            "side_detector": {"num_lasers": 0},
            "lane_line_detector": {"num_lasers": 0},
            "image_source": "rgb"
        },
        "sensors": {
            "rgb": (RGBCamera, width, height)
        },
        "traffic_density": 0.05,
        "num_scenarios": 1,
        "start_seed": 42,
        "window_size": (1200, 800),
        "show_interface": True,
        "show_fps": True
    }
    
    env = MetaDriveEnv(env_config)
    obs, _ = env.reset()
    
    # Process initial observation to determine input channels
    state = preprocess_observation(obs)
    input_channels = state.shape[0]  # Usually 3 for RGB
    
    num_actions = 5
    global config
    config = PPOConfig()
    
    # Initialize model and optimizer
    model = CNN_ActorCritic(input_channels, num_actions, height, width).to(config.device)
    optimizer = optim.Adam(model.parameters(), lr=config.lr, eps=config.eps)
    
    # Data structure for rollouts
    Transition = namedtuple('Transition', ['state', 'action', 'log_prob', 'reward', 'value', 'done'])
    
    for episode in range(config.num_episodes):
        obs, _ = env.reset()
        state = preprocess_observation(obs)
        total_reward = 0
        rollouts = []
        
        for step in range(config.max_steps):
            env.render()
            
            # Select action
            action, log_prob, value = model.act(state)
            
            # Map discrete action to continuous
            act = {
                0: [0.0, 1.0, 0.0],  # Forward
                1: [0.0, 0.0, 0.0],  # Coast
                2: [-0.5, 1.0, 0.0], # Left
                3: [0.5, 1.0, 0.0],  # Right
                4: [0.0, 0.0, 1.0]   # Brake
            }[action]
            
            # Step environment
            try:
                obs, reward, terminated, truncated, _ = env.step(act)
                next_state = preprocess_observation(obs)
                done = terminated or truncated
            except Exception as e:
                print(f"Error during step: {e}")
                done = True
                reward = 0
                next_state = state
            
            # Store transition
            rollouts.append(Transition(state, action, log_prob, reward, value, done))
            state = next_state
            total_reward += reward
            
            if done:
                break
        
        # Process rollouts
        if not rollouts:
            continue
        
        states = torch.stack([t.state for t in rollouts])
        actions = torch.tensor([t.action for t in rollouts], dtype=torch.long).to(config.device)
        old_log_probs = torch.tensor([t.log_prob for t in rollouts], dtype=torch.float32).to(config.device)
        rewards = [t.reward for t in rollouts]
        values = [t.value for t in rollouts]
        dones = [t.done for t in rollouts]
        
        # Compute GAE
        with torch.no_grad():
            _, next_value = model(states[-1].unsqueeze(0))
            next_value = next_value.item()
        
        advantages = compute_gae(rewards, values, next_value, dones, config.gamma, config.lam)
        advantages = torch.tensor(advantages, dtype=torch.float32).to(config.device)
        returns = advantages + torch.tensor(values, dtype=torch.float32).to(config.device)
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        
        # PPO Update
        for _ in range(config.ppo_epochs):
            indices = np.arange(len(rollouts))
            np.random.shuffle(indices)
            for start in range(0, len(rollouts), config.batch_size):
                end = start + config.batch_size
                batch_indices = indices[start:end]
                
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_old_log_probs = old_log_probs[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]
                
                # Forward pass
                probs, values = model(batch_states)
                dist = torch.distributions.Categorical(probs)
                log_probs = dist.log_prob(batch_actions)
                entropy = dist.entropy().mean()
                
                # Compute ratios
                ratios = torch.exp(log_probs - batch_old_log_probs)
                
                # Policy loss
                surr1 = ratios * batch_advantages
                surr2 = torch.clamp(ratios, 1 - config.clip_param, 1 + config.clip_param) * batch_advantages
                policy_loss = -torch.min(surr1, surr2).mean()
                
                # Value loss
                value_loss = F.mse_loss(values.squeeze(), batch_returns)
                
                # Total loss
                loss = policy_loss + config.value_loss_coef * value_loss - config.entropy_coef * entropy
                
                # Optimize
                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                optimizer.step()
        
        print(f"Episode {episode + 1}/{config.num_episodes}, Total Reward: {total_reward:.2f}")

    env.close()

if __name__ == "__main__":
    train_ppo()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), rgb: RGBCamera(84, 84), main_camera: MainCamera(1200, 800), dashboard: DashBoard()]
[INFO] Render Mode: onscreen
[INFO] Horizon (Max steps per agent): 1000
[WARNING] You have set norm_pixel = False, which means the observation will be uint8 values in [0, 255]. Please make sure you have parsed them later before feeding them to network! (metadrive_env.py:117)
[INFO] Assets version: 0.4.3
[INFO] Known Pipes: CocoaGraphicsPipe
2025-05-01 17:51:13.191 Python[81201:33646526] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-01 17:51:13.191 Python[81201:33646526] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[INFO] Start Scenario Index: 42, Num Scenarios : 1


AttributeError: module 'direct.showbase.DConfig' has no attribute 'device'